# Index Funds from IronIA BigData

Investigating available index funds in IronIA Big Data. Used for the competition during the Master and before I decided to use ETFs for the investment algorithm. Described in chapter 3.4.

In [5]:
import pandas as pd
import numpy as np
from time import time
from matplotlib import pyplot as plt
from google.cloud import bigquery
from google.oauth2 import service_account

In [6]:
credentials = service_account.Credentials.from_service_account_file('harazinlt.json')
client = bigquery.Client(credentials=credentials)

### Exploring all available funds

In [7]:
sql = """SELECT isin,allfunds_id,short_name,manager_name,
            class_code,geo_zone,currency,inception_at,modified_date,
            management_fee,ongoing_charges,performance_fee
        FROM Ironia_Ext.instruments 
        WHERE available_ironia=True AND asset_type = 'RVG'
            AND nomin=False AND income=False
            --AND currency='EUR' AND currency_hedge=False 
        """
all_funds_RVG_EUR = client.query(sql).to_dataframe()
all_funds_RVG_EUR.index = all_funds_RVG_EUR['isin']
all_funds_RVG_EUR.head()

,isin,allfunds_id,short_name,manager_name,class_code,geo_zone,currency,inception_at,modified_date,management_fee,ongoing_charges,performance_fee
isin,,,,,,,,,,,,
IE0005304443,IE0005304443,0431025,"PIMCO STOCKS PLUS ""INV""",PIMCO,INV,US,USD,1999-01-07,2023-06-30,0.55,0.90,0.0
IE0002459539,IE0002459539,0431024,"PIMCO GIS STOCKS PLUS ""I""",PIMCO,I,US,USD,1998-12-31,2023-06-30,0.55,0.55,0.0
IE00B1D7YP71,IE00B1D7YP71,0021713,"PIMCO STOCKS PLUS ""E"" (USD)",PIMCO,E,US,USD,2006-09-11,2023-06-30,1.45,1.45,0.0
IE00B7W3YB45,IE00B7W3YB45,0170150,"PIMCO GIS STOCKSPLUS ""I"" (EURHDG)",PIMCO,I,US,EUR,2012-09-28,2023-06-30,0.55,0.55,0.0
IE00BKRC1166,IE00BKRC1166,0249521,"PIMCO STOCKS PLUS ""E"" (EURHDG)",PIMCO,E,US,EUR,2017-08-02,2023-06-30,1.45,1.45,NaN


In [8]:
# Filtering funds by geo_zone
all_funds_RVG_EUR.loc[(all_funds_RVG_EUR['geo_zone'] == 'FR') & (all_funds_RVG_EUR['ongoing_charges'] < 2) & (all_funds_RVG_EUR['inception_at'] < '2012-07-01')]

,isin,allfunds_id,short_name,manager_name,class_code,geo_zone,currency,inception_at,modified_date,management_fee,ongoing_charges,performance_fee
isin,,,,,,,,,,,,
FR0007065743,FR0007065743,0282252,"SYCOMORE FRANCECAP ""A"" (EUR)",SYCOMORE ASSET MANAGEMENT (FR),A,FR,EUR,2001-10-30,2023-06-30,1.5,1.5,20.0
FR0010111724,FR0010111724,0282253,"SYCOMORE FRANCECAP ""I"" (EUR)",SYCOMORE ASSET MANAGEMENT (FR),I,FR,EUR,2001-10-30,2023-06-30,1.0,1.0,20.0


### Exploring various ratios

In [9]:
sql = """SELECT distinct ratio
        FROM Ironia_Ext.Ratios 
        WHERE allfunds_id = 0018110 AND date = '2023-08-16'
        ORDER BY ratio"""
ratios = client.query(sql).to_dataframe()
ratios

,ratio
0,alpha
1,bear_beta
2,beta
3,bull_beta
4,calmar
5,correlation
6,downside_deviation
7,information_ratio
8,linear_return
9,log_annualized_return


In [10]:
sql = """SELECT distinct benchmark_name
        FROM Ironia_Ext.Ratios AS R
                INNER JOIN Ironia_Ext.instruments AS I ON R.allfunds_id = CAST(I.allfunds_id AS INT)
        WHERE R.ratio = 'tracking_error' AND R.period = '5y' AND R.date = '2023-09-20' AND R.value_product < 0.1 AND
                available_ironia=True AND nomin=False AND income=False 
                --AND currency='EUR' AND currency_hedge=False 
                AND management_fee < 1 AND asset_type = 'RVG'
        ORDER BY benchmark_name
        """
ratios = client.query(sql).to_dataframe()
ratios

,benchmark_name
0,DAX
1,EURO STOXX 50 Net Return EUR
2,FTSE 100 Net Tax Index
3,FTSE MIB Net Total Return Index
4,Ibex 35
5,MSCI AC ASEAN USD Net
6,MSCI AC Asia Pacific Net Total Return USD Index
7,MSCI AC Asia ex Japan Net Total Return USD Index
8,MSCI ACWI Growth Net Total Return USD Index
9,MSCI ACWI Net Total Return USD Index


In [11]:
benchmark = 'MSCI ACWI Net Total Return USD Index'

sql = f"""SELECT isin, short_name, manager_name, benchmark_name,
            geo_zone, currency, inception_at,
            management_fee, ongoing_charges, --performance_fee,
            R.allfunds_id, R.value_product
        FROM Ironia_Ext.Ratios AS R
                INNER JOIN Ironia_Ext.instruments AS I ON R.allfunds_id = CAST(I.allfunds_id AS INT)
        WHERE R.ratio = 'tracking_error' AND R.period = '3y' AND R.date = '2023-09-20' AND R.value_product < 0.1 AND
                available_ironia=True AND nomin=False AND income=False AND currency='EUR' AND currency_hedge=False 
                AND management_fee < 1 AND ongoing_charges < 2 AND (performance_fee IS NULL OR performance_fee = 0) 
                AND benchmark_name = '{benchmark}'
        ORDER BY R.value_product
        LIMIT 10
        """
low_tracking_error = client.query(sql).to_dataframe()
low_tracking_error

,isin,short_name,manager_name,benchmark_name,geo_zone,currency,inception_at,management_fee,ongoing_charges,allfunds_id,value_product
0,LU1559926461,"EURIZON EF SUSTAINABL ""X"" (EUR)",EURIZON CAPITAL S.A.,MSCI ACWI Net Total Return USD Index,GLB,EUR,2019-10-10,0.750,0.92,96006,0.039762
1,LU0941351339,"UBS (LUX) STRATEGY FU ""Q"" (EUR) B",UBS GLOBAL AM,MSCI ACWI Net Total Return USD Index,GLB,EUR,2013-12-19,0.920,1.20,255955,0.050281
2,LU0942290494,"CANDRIAM QUANT EQ MF GLOBAL ""R"" (EU) ACC",CANDRIAM LUX (CACEIS),MSCI ACWI Net Total Return USD Index,GLB,EUR,2018-07-02,0.500,0.80,228161,0.071374
3,LU1240795606,"UBS (LUX) KEY SELECTI ""Q"" (EUR)",UBS GLOBAL AM,MSCI ACWI Net Total Return USD Index,GLB,EUR,2015-10-01,0.820,1.10,273821,0.078318
4,LU1902337663,"UBS (LUX) EQUITY FUND ""Q"" (EUR)",UBS GLOBAL AM,MSCI ACWI Net Total Return USD Index,GLB,EUR,2018-11-14,0.820,1.10,101238,0.082042
5,LU0996181599,"AMUNDI INDEX MSCI WORLD ""IE"" (EUR) ACC",AMUNDI FUNDS,MSCI ACWI Net Total Return USD Index,GLB,EUR,2016-06-29,0.075,0.20,242965,0.082956
6,LU0996182720,"AMUNDI INDEX MSCI WORLD ""RE"" (EUR) ACC",AMUNDI FUNDS,MSCI ACWI Net Total Return USD Index,GLB,EUR,2016-06-29,0.075,0.25,242971,0.082974
7,LU0996182563,"AMUNDI INDEX MSCI WORLD ""AE"" (EUR) ACC",AMUNDI FUNDS,MSCI ACWI Net Total Return USD Index,GLB,EUR,2016-06-29,0.125,0.30,239959,0.082982
8,IE00B62WCL09,"ISHARES DEV WRD INDX (IE) ""I"" (EUR) ACC",BLACKROCK INDEX SELECTION FUND,MSCI ACWI Net Total Return USD Index,GLB,EUR,2010-05-19,0.150,0.16,88006,0.083322
9,IE00B03HD191,VANGUARD GLOBAL STOCK INDEX (EUR) ACC,VANGUARD INVESTMENT SERIES PLC,MSCI ACWI Net Total Return USD Index,GLB,EUR,2002-12-10,0.180,0.18,14636,0.083475


### Selected funds - optimized for fees and index tracking

In [12]:
country_to_isin = {
    'United States': 'IE0032126645', # 2002
    'Japan': 'IE0007286036', # 2006
    'United Kingdom': 'IE00B1W56R86', # in GBP 2011
    'Canada': 'LU1419797797', # 2016
    'Switzerland': 'LU0546268359', # 2010
    'Germany': 'LU0740823785', # 2012
    'Australia': 'LU0256781096', # 2007
    'Spain': 'ES0149051007',# 2016
    'Italy': 'LU0087656855', # 1999
    'Eurozone': 'IE0008248803', # 2001
    'China': 'LU0625737910', # 2011
    'India': 'LU0248271941', # 2006
    'Korea': 'LU0301637293', # 2007
    'Brazil': 'LU0616857586', # 2012
    'Turkiye': 'LU0823433775', # 2013
    'Russia': 'LU0399027704', # in USD 2011 to 2022-02
    'ACWI': 'IE00B62WCL09' # benchmark ACWI Index
} 

selected_funds = all_funds_RVG_EUR.loc[country_to_isin.values()]
selected_funds

,isin,allfunds_id,short_name,manager_name,class_code,geo_zone,currency,inception_at,modified_date,management_fee,ongoing_charges,performance_fee
isin,,,,,,,,,,,,
IE0032126645,IE0032126645,0018110,VANGUARD U.S. 500 STOCK INDEX (EUR) ACC,VANGUARD INVESTMENT SERIES PLC,I,US,EUR,2002-09-26,2023-06-30,0.10,0.10,NaN
IE0007286036,IE0007286036,0067444,VANGUARD JAPAN STOCK INDEX (EUR) ACC,VANGUARD INVESTMENT SERIES PLC,A,JP,EUR,2006-03-29,2023-06-30,0.16,0.16,NaN
IE00B1W56R86,IE00B1W56R86,0089824,"ISHARES UK INDEX ""I"" (GBP) ACC",BLACKROCK INDEX SELECTION FUND,I,UK,GBP,1998-12-31,2023-06-30,0.15,0.17,NaN
LU1419797797,LU1419797797,0002602,"CSIF (LUX) EQUITY CAN ""QB"" (EUR)",CS INVESTMENT FUNDS,QB,CA,EUR,2016-06-14,2023-06-30,0.15,0.10,NaN
LU0546268359,LU0546268359,0086151,UBS (LUX) EQ - SWISS OPP P (EUR) ACC,UBS GLOBAL AM,P,CH,EUR,2010-11-16,2023-06-30,1.20,1.60,0.0
LU0740823785,LU0740823785,0204660,"DWS INVEST GERMAN EQ ""FC"" (EUR) ACC",DWS INVESTMENTS SA,FC,DE,EUR,2012-08-20,2023-06-30,0.75,0.85,NaN
LU0256781096,LU0256781096,0051017,"CANDRIAM EQS L AUSTRALIA ""I"" (EUR)",CANDRIAM LUX (CACEIS),I,AU,EUR,2007-03-26,2023-06-30,0.55,0.75,NaN
ES0149051007,ES0149051007,0001821,"ABANCA IBEX 35 ""D"" (EUR)","ABANCA GESTION DE ACTIVOS, S.G.I.I.C",D,ES,EUR,2016-06-10,2023-06-30,0.35,0.47,NaN
LU0087656855,LU0087656855,0015646,"AXA WF ITALY EQUITY ""F"" (EUR)",AXA INVESTMENT MANAGERS,F,IT,EUR,1998-05-01,2023-06-30,0.75,1.00,0.0


### Retrieving historical prices since 2013

In [13]:
sql = """SELECT date, nav 
        FROM Ironia_Ext.Navs 
        WHERE allfunds_id = 0088006
        ORDER BY date"""
prices = client.query(sql).to_dataframe()
prices

,date,nav
0,2011-01-26,10.340000
1,2011-02-03,10.499000
2,2011-02-15,10.695895
3,2011-02-16,10.754890
4,2011-02-17,10.767325
...,...,...
3008,2023-08-16,36.411228
3009,2023-08-17,36.178465
3010,2023-08-18,36.177785
3011,2023-08-21,36.292887


In [14]:
df_prices = pd.DataFrame(index=pd.date_range('1999-01-04', '2023-08-23', freq='B'))
for country in country_to_isin:
    isin = country_to_isin[country]
    allfunds_id = selected_funds.loc[isin].loc['allfunds_id']

    sql = f"""SELECT * 
            FROM Ironia_Ext.Navs 
            WHERE allfunds_id = {allfunds_id}
            ORDER BY date"""
    prices = client.query(sql).to_dataframe()
    prices.index = pd.DatetimeIndex(prices['date'])
    prices = prices[~prices.index.duplicated(keep='first')]
    df_prices[country] = prices['nav']

df_prices

,United States,Japan,United Kingdom,Canada,Switzerland,Germany,Australia,Spain,Italy,Eurozone,China,India,Korea,Brazil,Turkiye,Russia,ACWI
1999-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-17,47.7749,233.0415,19.870166,1649.46,289.62,223.48,283.35,13.599013,285.91,263.7527,107.28,44.00,12.86,176.37,255.58,NaN,36.178465
2023-08-18,47.8402,233.1748,19.748889,1645.97,287.67,220.67,283.76,13.582803,284.40,262.6807,104.89,44.00,12.84,175.44,247.63,NaN,36.177785
2023-08-21,48.1075,231.4313,19.732059,NaN,287.85,222.43,282.85,13.574787,285.18,263.2737,102.87,44.15,12.79,174.38,255.67,NaN,36.292887
2023-08-22,48.1377,235.6071,19.751298,1639.32,290.00,223.19,284.51,13.650714,287.10,265.2265,104.25,44.50,12.97,177.45,256.04,NaN,36.399962


In [15]:
df_prices = df_prices.ffill(limit=5)
df_prices.to_csv('Index_Funds.csv')

In [16]:
for country in country_to_isin:
    first_valid = df_prices[country].first_valid_index()
    print(country, first_valid)

United States 2002-09-26 00:00:00
Japan 2006-03-29 00:00:00
United Kingdom 2011-06-27 00:00:00
Canada 2016-06-14 00:00:00
Switzerland 2010-11-16 00:00:00
Germany 2012-08-20 00:00:00
Australia 2007-03-26 00:00:00
Spain 2016-04-29 00:00:00
Italy 1999-01-05 00:00:00
Eurozone 2001-10-25 00:00:00
China 2011-07-13 00:00:00
India 2006-04-27 00:00:00
Korea 2007-10-02 00:00:00
Brazil 2012-10-24 00:00:00
Turkiye 2013-05-22 00:00:00
Russia 2011-11-11 00:00:00
ACWI 2011-01-26 00:00:00


In [17]:
for country in country_to_isin:
    last_valid = df_prices[country].last_valid_index()
    if not str(last_valid).startswith('2023-08'):
        print(country, last_valid)

Russia 2022-03-03 00:00:00
